## The wonderful 1 dimensional bandstructure function
 **Autors:** *Klára Nováková* and *Andris Potrebko* with a vital teoretical support of *Andreas Wacker*, *Tor Sjöstrand* and *Hofmanns* book *Solid State Physics*
 **Year:** *2019*
 
### Some descrption is gonna come here

#### All of the plots at the moment are not pretty and stupid. The Plotting outside the first Brilluin zone is tricky is has to delt later

In [19]:
%matplotlib notebook 
# Usual imports
import matplotlib.pyplot as plt
import numpy as np

# pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Widgets imports
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
from ipywidgets import HBox, Label
from ipywidgets import IntSlider

#### Summary from all of the different notes we have:

We have the Gaussian potential $V(x)=-U\sum_n e^{-(x-na)^2/2\sigma^2}$ which is writen in Fourier components as: $V(x)=\sum_{G_j}-U\sqrt {2\pi}\frac{\sigma}{a}e^{-G^2_j \sigma^2/2} e^{i G_j x} \equiv  \sum_{G_j} V_{G_j} \cdot e^{i G_j x}$.

We are solving the eigenvalue equation:
$\Big( \frac{\hbar^2 (k+K)^2}{2m} - \varepsilon \Big) c_{k+K} +\sum_G c_{k+K-G} V_G$ and search for $\varepsilon$ - the energy eigenvalues.

Define different stuff
* Don't confuse, *K* is a reciprocal lattice vector, *k* is a wave vector $k\in [-\pi/a; \pi/a)$ (number of *k* vectors is *Len_k*)
* *MaxK* - maximal reciprocal lattice vector we consider including in the sum ($K$ or $K-G$). Then summation goes from *-MaxK* to *MaxK*

### Define some initial values. Interactive part will later change these variables

In [20]:
a=3   #Latice constant in Åmstrongs
sigma=0.5 #Very arbitrary value
U=11     #eV
maxK=6  
Len_k=15

NrOfK=maxK*2+1

The ampliture of $V_G$ we call $A$, thus $A \equiv -U\sqrt {2\pi}\frac{\sigma}{a}$ and  $V_G = A e^{-G^2 \sigma^2/2} $

In [21]:
A=-U*np.sqrt(2 *np.pi)*sigma/a

In [23]:
np.exp(-0.5)

0.6065306597126334

*BandStructureFunction.ipynb* file contains a function *BandStructF* which creates the matrix described above and solves for the eigenvalues.
*%run* does import of the function found in the file

In [4]:
%run BandStructureFunction.ipynb

Solve eigenvalue equation for each wave number *k*
* Put k vector into right magnitude.  Before *k* is some integer $k =.. -2,-1,0,1..$. Save the obtained value in *kVect*
* Save calculated energies in *Energies*

In [5]:
%%time
Energies=np.zeros(shape=(Len_k*2+1, NrOfK))
kVect=[0]*(Len_k*2+1) #Vector for plotting containing all of the k values in 1/Angstrom
for ki in range(-Len_k,Len_k+1):
    k=np.pi/(a)*ki/Len_k
#     %run BandStructureFunction.ipynb
    E = BandStructF()
    Energies[ki+Len_k]=np.real(E)
    kVect[ki+Len_k]=k #in 1/Angstrom

Wall time: 100 ms


## Plots

### Plot potential

Plots are hidden so that they would not appear when running the interface. To make the plots visible either comment out the lines containg *%%capture* or comment out the cell containing *fig*

In [6]:
%%capture 
#If you want to display the figure here, then uncomment previous line
from matplotlib import pyplot as plt
%matplotlib notebook 
%matplotlib notebook 
fig, [ax,ax2]=plt.subplots(nrows=1, ncols=2)  #Two plots in the same figure
fig.set_size_inches(9.5, 3.5)
fig.subplots_adjust(wspace=0.4, bottom=0.2) #Margins around the subplots


x=np.linspace(-5*a,5*a,401)  
#length of the x vector should be an odd number so that 0 is included,
#otherwise it does not plot the peak correctly
Vx=-U*np.exp(-x**2/(2*sigma**2))
VxSum=[0]*len(x)
for n in range(-5,6):
    Vx1=-U*np.exp(-(x-n*a)**2/(2*sigma**2))
    ax.plot(x,Vx1,'y--')
    VxSum=VxSum+Vx1
ax.plot(x,VxSum)
ax.plot(x,Vx)
ax.set(xlabel='a, ($\AA$)',ylabel='V (eV)', title='Atomic potential')

### Plot energies

In [7]:
%%capture
ax2.plot(kVect,Energies[:,0:5], color='purple')
ax2.set(xlabel='k ($1/ \AA$)',ylabel='E (eV)', title='Band structure')

In [8]:
# fig

### The interactive part starts here

Function *f* is going to repeat everything above whenever some of the widgets are going to be used. Function *interactive* is making these widgets and connecting them to the function *f*.

In [9]:
def f(Lattice_Constant=a, Amplitude=U, NumKVec=maxK, PlotK=5):
    global a,A, k, Len_k, maxK, U, NrOfK  #So that BandStructureFunction.ipynb knows the variables used in a function f
    a=Lattice_Constant
    U=Amplitude
    maxK=NumKVec
    A=-U*np.sqrt(2 *np.pi)*sigma/a #np.sqrt(2*sigma) 
    NrOfK=maxK*2+1
    Energies=np.zeros(shape=(Len_k*2+1, NrOfK))
    kVect=[0]*(Len_k*2+1)
    for ki in range(-Len_k,Len_k+1):
        k=np.pi/(a)*ki/Len_k
        E = BandStructF()
#         %run BandStructureFunction.ipynb
        Energies[ki+Len_k]=np.real(E)
        kVect[ki+Len_k]=k
    ax.cla()
    ax.set(xlabel='a, ($\AA$)',ylabel='V (eV)', title='Atomic potential')
    x=np.linspace(-5*a,5*a,401)
    Vx=-U*np.exp(-x**2/(2*sigma**2))
    VxSum=[0]*len(x)
    for n in range(-5,6):
        Vx1=-U*np.exp(-(x-n*a)**2/(2*sigma**2))
        ax.plot(x,Vx1,'y--')
        VxSum=VxSum+Vx1
        
  
    ax.plot(x,VxSum)
    ax.plot(x,Vx)
   
    ax2.cla()
    ax2.set(xlabel='k ($1/ \AA$)',ylabel='E (eV)', title='Band structure')
    ax2.plot(kVect, Energies[:,0:PlotK],color='purple')

    #return Energies

The_Interaction=interactive(f,Lattice_Constant=(1,10,0.5),Amplitude=(0,100,1), NumKVec=(1,20,1),PlotK=(0,10,1) ) 

### Create boxed for widgets to be displayed

In [10]:
for widg in The_Interaction.children[:-1]:
    widg.description = ""
    widg.continuous_update = False
    

Lattice_Const, Potential_Amp, Num_Of_K, PlotK = [The_Interaction.children[i] for i in range(4)]

#display(Lattice_Const,Potential_Amp ,Num_Of_K )
FirstBox = widgets.HBox([Label(r'Lattice constant (Å)'), Lattice_Const, Label(r'Potential depth, (eV)'), Potential_Amp,])
SecondBox = widgets.HBox([ Label (r'N of K vectors'), Num_Of_K, Label (r'How many bands to plot'), PlotK])

That, how many bands we have calculated depends *maxK* as $2\cdot maxK + 1$. So we make that the maximal possible band to plot changes accordingly

In [11]:
def PlotKMax(*args):
    PlotK.max = 2*Num_Of_K.value+1

Num_Of_K.observe(PlotKMax, 'value')

We can calculate the theoretical minimum of the first band

#Trying to check something

In [24]:
V0=U*5  *1.9e-19 #eV, potential depth
E0 = hbar/sigma*np.sqrt(V0/(m_e)) *1e10/(1.9e-19)

Some attempt checking using that near the Brilluin zone boundary $E_{1,2} \approx \frac{\hbar^2}{4m}(2k^2 - 2kg +g^2) \pm \sqrt{\frac{\hbar^4}{16m^2}(k^2 - (k-g)^2)^2 + U^2}$
that is proven in Solid Stete theory course

In [13]:
E2 = NormC*(np.pi/a)**2 -np.sqrt( (NormC*(np.pi/a)**2)**2 + U**2  )

E1 = NormC*(np.pi/a)**2 + np.sqrt( (NormC*(np.pi/a)**2)**2 + U**2  )

In [25]:
E0

37.5981260341162

In [14]:
E2
E1

-7.588646540495164

15.944872297623794

In [15]:
E1+E2

8.35622575712863

In [16]:
K=0
ki=15
k = np.pi/(a)*ki/Len_k
NormC*(k+K)**2 + A

-0.41737229159251843

In [17]:
k

1.0471975511965976

In [18]:
A

-4.5954851701568336